![Synthesized Banner](./synthesized_banner.png)

# Train, tune, and deploy a custom synthetic data generator using Synthesized's Tabular Data Generator Algorithm from AWS Marketplace


[Synthesized's Tabular Data Generator Algorithm]() brings the generative AI capabilitiies of our [SDK](https://docs.synthesized.io/sdk/latest/) to AWS Sagemaker.

## Overview

Our Scientific Data Kit (SDK) is a comprehensive framework for generative modelling for structured data. It helps you create compliant statistical-preserving data snapshots for BI/Analytics and ML/AI applications. Right-size your data with AI-supported data transformations.

You can find more details in our [documentation](https://docs.synthesized.io/sdk/latest/).

This sample notebook shows you how to train a custom ML model using [Synthesized's Tabular Data Generator Algorithm]() from AWS Marketplace.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Synthesized's Tabular Data Generator Algorithm](). 

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize train and test dataset](#B.-Configure-and-visualize-train-and-test-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model and verify results](#4:-Deploy-model-and-verify-results)
    1. [Deploy trained model](#A.-Deploy-trained-model)
    1. [Create input payload](#B.-Create-input-payload)
    1. [Perform real-time inference](#C.-Perform-real-time-inference)
    1. [Visualize output](#D.-Visualize-output)
    1. [Calculate relevant metrics](#E.-Calculate-relevant-metrics)
    1. [Delete the endpoint](#F.-Delete-the-endpoint)
1. [Tune your model! (optional)](#5:-Tune-your-model!-(optional))
	1. [Tuning Guidelines](#A.-Tuning-Guidelines)
	1. [Define Tuning configuration](#B.-Define-Tuning-configuration)
	1. [Run a model tuning job](#C.-Run-a-model-tuning-job)
1. [Perform Batch inference](#6.-Perform-Batch-inference)
1. [Clean-up](#7.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page [Synthesized's Tabular Data Generator Algorithm]().
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
# algo_arn = "<Customer to specify algorithm ARN corresponding to their AWS region>"
algo_arn = "arn:aws:sagemaker:eu-west-2:248441125472:algorithm/synthesized-sdk-tabular-v2-8-48"

## 2. Prepare dataset

In [ ]:
!pip install insight==0.9rc6 numpy pandas

In [ ]:
import os
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
import urllib.request
import numpy as np
import pandas as pd
from insight import plot

### A. Dataset format expected by the algorithm

<font color='red'>For Seller to update: In following cell, provide a description of the dataset accepted by the training job. This will help customer understand what the dataset fed to the algorithm needs to look like. <font>

You can also find more information about dataset format in **Usage Information** section of [Synthesized's Tabular Data Generator Algorithm]().

### B. Configure and visualize train and test dataset

In [ ]:
training_dataset = "data/algorithm/train/credit_orig.csv"
training_data_dir = os.path.dirname(training_dataset)

In [ ]:
df = pd.read_csv(training_dataset)
plot.dataset([df], figure_cols=4, max_categories=20)
df.info()

In [ ]:
training_config = "data/algorithm/config/credit_config.json"
training_config_dir = os.path.dirname(training_config)

In [ ]:
with open(training_config, 'rb') as file:
    config = json.load(file)
print(json.dumps(config, indent=2))

### C. Upload datasets to Amazon S3

In [ ]:
sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
prefix = f"synth-{uuid.uuid4()}"

training_dataset_dir = sagemaker_session.upload_data(
    training_data_dir, bucket=bucket, key_prefix=f"{prefix}/train"
)
training_config_dir = sagemaker_session.upload_data(
    training_config_dir, bucket=bucket, key_prefix=f"{prefix}/config"
)

print(training_dataset_dir)
print(training_config_dir)

## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [ ]:
role = get_execution_role()

In [ ]:
output_location = f"s3://{bucket}/{prefix}/output"
print(output_location)

### 3.2 Train a model

You can also find more information about dataset format in **Hyperparameters** section of the [Tabular Synthesizer Algorithm]().

In [ ]:
# Define hyperparameters
hyperparameters = {
  "batch_size": "256",
  "latent_size": "32"
}

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [ ]:
import sagemaker
from sagemaker import AlgorithmEstimator

session = sagemaker.Session()

instance_type = 'ml.m5.2xlarge'

# Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="tabular-synthesizer",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,

)
# Run the training job.
estimator.fit({"train": training_dataset_dir, "config": training_config_dir})

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

<font color='red'>For seller to update: Update appropriate values in following cell. </font>

In [ ]:
model_name = "For Seller to update:<specify-model_or_endpoint-name>"
content_type = "For Seller to update:<specify_content_type_accepted_by_trained_model>"

real_time_inference_instance_type = 'ml.m5.2xlarge'
batch_transform_inference_instance_type = 'ml.m5.2xlarge'

### A. Deploy trained model

In [ ]:
predictor = estimator.deploy(
    1, real_time_inference_instance_type, serializer="<For seller to update>"
)

Once endpoint is created, you can perform real-time inference.

### B. Create input payload

<font color='red'>For Seller to update: Add code snippet that reads the input from 'data/inference/input/real-time/' directory 
and converts it into format expected by the endpoint in the following cell</font>

<Add code snippet that shows the payload contents>

<font color='red'>For Seller to update: Ensure that the `file_name` variable points to the payload you created. 
Ensure that the `output_file_name` variable points to a file-name in which output of real-time inference needs to be stored</font>

### C. Perform real-time inference

<font color='red'>For Seller to update: review/update `file_name`, `output_file name`, and custom attributes in the following AWS CLI example to perform a real-time inference using the payload file you created from 2.B </font>

In [ ]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

### D. Visualize output

<font color='red'>For Seller to update: Write code in the following cell to display the output generated by real-time inference. This output must match with output available in data/inference/output/real-time folder.</font>

### E. Calculate relevant metrics

<font color='red'>For seller to update: write code to calculate metrics such as accuracy or any other metrics relevant to the business problem, using the test dataset. **This is highly recommended if your algorithm does not support and calculate metrics on test channel**. For information on how to configure metrics for your algorithm, see [Step 4 of this blog post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/). </font>

If [Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) supports the type of problem you are trying to solve using this algorithm, use the following examples to add Model Monitor support to your product:
For sample code to enable and monitor the model, see following notebooks:
1. [Enable Amazon SageMaker Model Monitor](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_model_monitor/enable_model_monitor/SageMaker-Enable-Model-Monitor.ipynb)
2. [Amazon SageMaker Model Monitor - visualizing monitoring results](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_model_monitor/visualization/SageMaker-Model-Monitor-Visualize.ipynb)

### F. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. you can terminate the same to avoid being charged.

In [ ]:
predictor = sage.RealTimePredictor(model_name, sagemaker_session, content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

Since this is an experiment, you do not need to run a hyperparameter tuning job. However, if you would like to see how to tune a model trained using a third-party algorithm with Amazon SageMaker's hyperparameter tuning functionality, you can run the optional tuning step.

## 5: Tune your model! (optional)

<font color='red'>For Seller to update/read: It is important to provide hyperparameter tuning functionality as part of your algorithm. Users of algorithms range from new developers, to data scientists and ML practitioners. As an algorithm maker, you need to make your algorithm usable in production. To be able to do so, you need to give  tools such as capability to tune a custom ML model using Amazon SageMaker Automatic Model Tuning(HPO) SDK. Enabling your algorithm for automatic model tuning functionality is really easy. You need to mark appropriate hyperparameters as Tunable=True and emit multiple metrics that customers can choose to tune an ML model on.
    
    
We recommend that you provide notes on how your customer can scale usage of your algorithm for really large datasets. 
    
**You are strongly recommended to provide this section with tuning guidelines and code for running an automatic tuning job**.
    
For information about Automatic model tuning, see [Perform Automatic Model Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)

### A. Tuning Guidelines

<font color='red'>For Seller to update: Provide guidelines on how customer can tune their ML model effectively using your algorithm in following cell. Provide details such as which parameter can be tuned for best results.  </font>

### B. Define Tuning configuration

<font color='red'>For seller to update: Provide a recommended hyperparameter range configuration in the following cell. This configuration would be used for running an [HPO](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) job. For More information, see [Define Hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)</font>

In [ ]:
hyperparameter_ranges = {}

<font color='red'>For seller to update: As part of your algorithm, provide multiple objective metrics so that customer can choose a metric for tuning 
a custom ML model. Update the following variable with a most suitable/popular metric that your algorithm emits. For more information, see [Define Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-metrics.html) </font>

In [ ]:
objective_metric_name = (
    "<For seller to update : Provide an appropriate objective metric emitted by the algorithm>"
)

<font color='red'>For seller to update: Specify whether to maximize or minimize the objective metric, in following cell.</font>

In [ ]:
tuning_direction = "<For seller to update: Provide tuning direction for objective metric specified>"

### C. Run a model tuning job

<font color='red'>For seller to update: Review/update the tuner configuration including but not limited to `base_tuning_job_name`, `max_jobs`, and `max_parallel_jobs`. </font>

In [ ]:
tuner = HyperparameterTuner(
    estimator=estimator,
    base_tuning_job_name="<For Seller to update: Specify base job name>",
    objective_metric_name=objective_metric_name,
    objective_type=tuning_direction,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=50,
    max_parallel_jobs=7,
)

<font color='red'>For seller to update: Uncomment following lines, specify appropriate channels, and run the tuner to test it out. </font>

In [ ]:
# Uncomment following two lines to run Hyperparameter optimization job.
# tuner.fit({'training':  data})
# tuner.wait()

<font color='red'>For seller to update: Once you have tested the code written in the preceding cell, comment three lines in the preceding cell so that customers who choose to simply run entire notebook do not end up triggering a tuning job. </font>

Once you have completed a tuning job, (or even while the job is still running) you can [clone and use this notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) to analyze the results to understand how each hyperparameter effects the quality of the model.

## 6. Perform Batch inference

In this section, you will perform batch inference using multiple input payloads together.

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/inference/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

<font color='red'>For Seller to update: Add code that displays output generated by the batch transform job available in S3.  This output must match the output available in data/inference/output/batch folder.</font>

## 7. Clean-up

### A. Delete the model

In [ ]:
predictor.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

